In [6]:
# Performs a simulation to come up with the number of games required to achieve each rank in a season of MTG Arena
# Using data from the Dec 18 State of the Beta

import random
import pandas as pd
MAX_GAMES = int((31 * 24 * 60) / 5) # A high end that is physically impossible for any one person to hit
WINRATE_INCREMENT = 0.01
SIMULATIONS = 1000

In [7]:
# Rank	Steps Gained with Win	Steps Lost with Loss	Steps needed to Advance a Tier
# Bronze	2	0	4
# Silver	2	1	5
# Gold	    1	1	6
# Platinum	1	1	7
# Diamond	1	1	7

# never get pushed to a lower rank

ranks = [
    {"title": "Bronze", "PerWin": 2, "PerLoss": 0, "PerTier": 4},
    {"title": "Silver", "PerWin": 2, "PerLoss": 1, "PerTier": 5},
    {"title": "Gold", "PerWin": 1, "PerLoss": 1, "PerTier": 6},
    {"title": "Platinum", "PerWin": 1, "PerLoss": 1, "PerTier": 7},
    {"title": "Diamond", "PerWin": 1, "PerLoss": 1, "PerTier": 7},
    {"title": "Mythic", "PerWin": 1, "PerLoss": 1, "PerTier": 99999999999999999999999}
]

class Player:
    def __init__(self, winrate):
        self.winrate = winrate
        self.rank = 0
        self.tier = 4
        self.steps = 0
    def getRank(self):
        return ranks[self.rank]
    def simGame(self):
        # Returns true for win false for loss
        return random.random() <= self.winrate
    def simRanked(self):
        rank = self.getRank()
        win = self.simGame()
        if win:
            self.steps += rank["PerWin"]
        else:
            self.steps -= rank["PerLoss"]

        if self.steps < 0:
            self.downgradeTier()
        elif self.steps >= rank["PerTier"]:
            self.upgradeTier()
    def upgradeTier(self):
        self.tier -= 1
        if self.tier <= 0:
            self.upgradeRank()
        self.steps = 0
    def downgradeTier(self):
        if self.tier == 4:
            # Can't be downgraded any more
            self.steps = 0
            return
        self.tier += 1
        self.steps = self.getRank()["PerTier"] - 1
    def upgradeRank(self):
        self.rank += 1
        self.steps = 0
        self.tier = 4

In [8]:
import time
start_time = time.time()
results = []
for w in range(0,100, int(WINRATE_INCREMENT*100)):
    winrate = w/100.
    tracker = {"Silver": [], "Gold": [], "Platinum": [], "Diamond": [], "Mythic": []}
    for _ in range(SIMULATIONS):
        player = Player(winrate)
        highestrank = player.rank
        for i in range(MAX_GAMES):
            player.simRanked()
            if player.rank > highestrank:
                highestrank = player.rank
                tracker[player.getRank()["title"]].append(i+1)
    r = {}
    for k,v in tracker.items():
        if len(v) == SIMULATIONS:
            # Player was able to achieve this rank 100% of the time
            r[k] = round(sum(v) / float(SIMULATIONS), 3)
        elif len(v) == 0:
            # Player was never able to achieve this rank
            r[k] = 'NA'
        else:
            # Player was sometimes able to achieve this rank, so list that probability
            prob = round(len(v) / float(SIMULATIONS), 2) * 100
            r[k] = "{} ({}%)".format(round(sum(v) / float(len(v)), 3), prob)
    r["Winrate"] = winrate
    results.append(r)
df_results = pd.DataFrame(results, columns=["Winrate", "Silver", "Gold", "Platinum", "Diamond", "Mythic"])
print("Took {} seconds".format(time.time() - start_time))

Took 812.6106288433075 seconds


In [5]:
pd.options.display.max_rows = 999
df_results.set_index("Winrate")

,Silver,Gold,Platinum,Diamond,Mythic
Winrate,,,,,
0.00,NA,NA,NA,NA,NA
0.01,743.7,NA,NA,NA,NA
0.02,401.2,NA,NA,NA,NA
0.03,288.3,NA,NA,NA,NA
0.04,238.4,NA,NA,NA,NA
0.05,111.5,NA,NA,NA,NA
0.06,116.7,NA,NA,NA,NA
0.07,116.9,NA,NA,NA,NA
0.08,112.1,NA,NA,NA,NA
